In [1]:
import mne
import numpy as np
import os
import pandas as pd
from os import listdir
from os.path import isfile, join
import sys
import math
# import pandas as pd
import scipy.io as sio
from sklearn import preprocessing
from scipy.signal import butter, lfilter
from scipy.io import loadmat

In [2]:
seg_size=200
channels=62
sub_bands=5

In [4]:
# data_read='data_address'
data_read= r'E:\EEG\SEED_Dataset\seed_analysis\Preprocessed_EEG'
file1 = os.listdir(data_read)
file1

['10_20131130.mat',
 '10_20131204.mat',
 '10_20131211.mat',
 '11_20140618.mat',
 '11_20140625.mat',
 '11_20140630.mat',
 '12_20131127.mat',
 '12_20131201.mat',
 '12_20131207.mat',
 '13_20140527.mat',
 '13_20140603.mat',
 '13_20140610.mat',
 '14_20140601.mat',
 '14_20140615.mat',
 '14_20140627.mat',
 '15_20130709.mat',
 '15_20131016.mat',
 '15_20131105.mat',
 '1_20131027.mat',
 '1_20131030.mat',
 '1_20131107.mat',
 '2_20140404.mat',
 '2_20140413.mat',
 '2_20140419.mat',
 '3_20140603.mat',
 '3_20140611.mat',
 '3_20140629.mat',
 '4_20140621.mat',
 '4_20140702.mat',
 '4_20140705.mat',
 '5_20140411.mat',
 '5_20140418.mat',
 '5_20140506.mat',
 '6_20130712.mat',
 '6_20131016.mat',
 '6_20131113.mat',
 '7_20131027.mat',
 '7_20131030.mat',
 '7_20131106.mat',
 '8_20140511.mat',
 '8_20140514.mat',
 '8_20140521.mat',
 '9_20140620.mat',
 '9_20140627.mat',
 '9_20140704.mat',
 'label.mat',
 'readme.txt']

In [6]:
# labels=loadmat('label_address')
# labels

In [7]:
################BANDPASS FILTER################
def butter_band(lowcut, highcut, fsamp, order=3):
    b, a = butter(order, [lowcut, highcut], btype='band', fs=fsamp)
    return b, a

def butter_BPF_filter(data, lowcut, highcut, fs, order=3):
    b, a = butter_band(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

###############COMPUTATION OF DIFFERENTIAL ENTROPY##########
def diff_ent_feat(signal):
    variance = np.var(signal, ddof=1)
    diff_ent = math.log(2 * math.pi * math.e * variance) / 2
    return diff_ent

In [47]:
def DE_feature_all(file):
    ############################
    data = loadmat(file)
    frequency = 200
    trials=[key for key in data.keys()]
    trials=trials[3:]
    DE_feat_matrix = np.empty([0, channels, sub_bands])
    #################
    for trial in range(2):
        tmp_signal_emot = data[trials[trial]]
        num_seg = int(len(tmp_signal_emot[0]) / seg_size)
        matrix_de1 = np.empty([0, num_seg ])
        #########################
        for channel in range(channels):
            signal_emot = tmp_signal_emot[channel]
            #########################
            del_band = butter_BPF_filter(signal_emot, 1, 4, frequency, order=3)
            th_band = butter_BPF_filter(signal_emot, 4, 8, frequency, order=3)
            alp_band = butter_BPF_filter(signal_emot, 8, 14, frequency, order=3)
            bet_band = butter_BPF_filter(signal_emot, 14, 31, frequency, order=3)
            gam_band = butter_BPF_filter(signal_emot, 31, 51, frequency, order=3)
            ##########################
            DE_del_band = np.zeros(shape=[0], dtype=float)
            DE_th_band = np.zeros(shape=[0], dtype=float)
            DE_alp_band = np.zeros(shape=[0], dtype=float)
            DE_bet_band = np.zeros(shape=[0], dtype=float)
            DE_gam_band = np.zeros(shape=[0], dtype=float)
            ########################
            for index in range(num_seg):
                de1 = diff_ent_feat(del_band[index * seg_size:(index + 1) * seg_size]) 
                DE_del_band = np.append(DE_del_band,de1)
                ######################
                de2 = diff_ent_feat(th_band[index * seg_size:(index + 1) * seg_size])
                DE_th_band = np.append(DE_th_band, de2)
                ######################
                de3 = diff_ent_feat(alp_band[index * seg_size:(index + 1) * seg_size])
                DE_alp_band = np.append(DE_alp_band, de3)
                ######################
                de4 = diff_ent_feat(bet_band[index * seg_size:(index + 1) * seg_size])
                DE_bet_band = np.append(DE_bet_band, de4)
                ######################
                de5 = diff_ent_feat(gam_band[index * seg_size:(index + 1) * seg_size])
                DE_gam_band = np.append(DE_gam_band, de5)
            #########################
            matrix_de = np.vstack([DE_del_band,DE_th_band,DE_alp_band,DE_bet_band,DE_gam_band])
#             print('matrix_de',matrix_de.shape)
            matrix_de1 = np.vstack([matrix_de1,matrix_de])           
        print('matrix_de1',matrix_de1.shape)
        matrix_de2 = matrix_de1.reshape(-1, sub_bands, num_seg)
        print('matrix_de2',matrix_de2.shape)
        matrix_de3 = matrix_de2.transpose([2, 0, 1])
        print('matrix_de3',matrix_de3.shape)
        DE_feat_matrix = np.vstack([DE_feat_matrix, matrix_de3])

    print("DE_feat_matrix shape:", DE_feat_matrix.shape)
    return DE_feat_matrix

######################################

In [48]:
# X = np.empty([0, channels, sub_bands])
# y = np.empty([0, 1])

In [51]:
# for i in range(len(file1)):
for i in range(1):
    file_name =  os.path.join(data_read,file1[i])
    print('processing {}'.format(file1[i]))
    DE_feat_matrix = DE_feature_all(file_name)
    print(f'DE_feat_matrix shape is :{DE_feat_matrix.shape}')
    subject=file1[i].split('.')
#     np.save('address_to_save_feat_matrix'+subject[0], DE_feat_matrix)

processing 10_20131130.mat
matrix_de1 (310, 235)
matrix_de2 (62, 5, 235)
matrix_de3 (235, 62, 5)
matrix_de1 (310, 233)
matrix_de2 (62, 5, 233)
matrix_de3 (233, 62, 5)
DE_feat_matrix shape: (468, 62, 5)
DE_feat_matrix shape is :(468, 62, 5)


In [52]:
y_all=np.empty([0])
all_label = [1, 0, -1, -1, 0, 1, -1, 0, 1, 1, 0, -1, 0, 1, -1]
samples =[235,233,206,238,185,195,237,216,265,237,235,233,235,238,206]
for i in range(15):
    y_all=np.append(y_all,all_label[i]*np.ones([samples[i]]))
np.save('address_to_save_labels', y_all)

In [53]:
DE_feat_matrix

array([[[4.45911106, 4.0614499 , 3.34315828, 3.20218052, 2.96804442],
        [4.34452332, 3.93729835, 3.33328566, 2.8174783 , 2.25684202],
        [4.16889644, 3.88309777, 3.24772429, 3.25443138, 3.16027276],
        ...,
        [3.06597253, 2.94435366, 2.42713594, 2.56145688, 2.07701913],
        [3.02191665, 3.08156544, 2.3987821 , 2.5634836 , 2.24392255],
        [3.05468218, 3.10931472, 2.37561827, 2.60411023, 2.2311039 ]],

       [[3.06985104, 2.36775608, 2.24129162, 2.95709164, 3.08472813],
        [3.1045043 , 2.48034482, 2.47004641, 2.46845065, 2.44493509],
        [3.06153051, 2.53144353, 2.4891678 , 3.01502726, 3.13759102],
        ...,
        [2.84596605, 2.15292461, 2.44409406, 2.49559967, 2.42701215],
        [2.83544554, 2.30262604, 2.57237092, 2.53897442, 2.37423934],
        [2.8596004 , 2.32743487, 2.56220645, 2.53633519, 2.36218186]],

       [[3.32776254, 3.09568854, 2.56427496, 2.89525191, 3.01046401],
        [3.21786178, 3.03427546, 2.51532812, 2.57171394, 2.5